In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [3]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = list(application_counts[application_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [4]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifi_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = list(classifi_counts[classifi_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [5]:
application_df.describe()

,STATUS,ASK_AMT,IS_SUCCESSFUL
count,34299.000000,3.429900e+04,34299.000000
mean,0.999854,2.769199e+06,0.532406
std,0.012073,8.713045e+07,0.498956
min,0.000000,5.000000e+03,0.000000
25%,1.000000,5.000000e+03,0.000000
50%,1.000000,5.000000e+03,1.000000
75%,1.000000,7.742000e+03,1.000000
max,1.000000,8.597806e+09,1.000000


In [6]:
# binning ASK_AMT
application_df['ASK_AMT'] = pd.cut(application_df['ASK_AMT'], bins=[0, 5000, 50000, 500000, 5000000, application_df.ASK_AMT.max()], labels=['0-5000','5000-50000', '50000-500000', '500000-5000000', 'above_5000000'] )
application_df['ASK_AMT'].value_counts()

0-5000            25398
50000-500000       3727
5000-50000         2947
500000-5000000     1590
above_5000000       637
Name: ASK_AMT, dtype: int64

In [7]:
dummy_df = pd.get_dummies(application_df)
dummy_df

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,ASK_AMT_0-5000,ASK_AMT_5000-50000,ASK_AMT_50000-500000,ASK_AMT_500000-5000000,ASK_AMT_above_5000000
0,1,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,1,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
2,1,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,1,0,0,0,0
3,1,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
4,1,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0
34295,1,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,1,0,0,0,0
34296,1,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
34297,1,1,0,0,0,0,0,1,0,0,...,0,0,0,1,0,1,0,0,0,0


In [8]:
# Split our preprocessed data into our features and target arrays
X = dummy_df.drop(columns='IS_SUCCESSFUL', axis=1).values
y = dummy_df['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [9]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Optimization

In [24]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation='relu', input_dim=len(X_train[0])))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 80)                3840      
                                                                 
 dense_5 (Dense)             (None, 30)                2430      
                                                                 
 dense_6 (Dense)             (None, 1)                 31        
                                                                 
Total params: 6,301
Trainable params: 6,301
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 4ms/step - loss: 0.5708 - accuracy: 0.7183
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5548 - accuracy: 0.7290
Epoch 3/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5506 - accuracy: 0.7309
Epoch 4/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5480 - accuracy: 0.7324
Epoch 5/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5471 - accuracy: 0.7334
Epoch 6/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5459 - accuracy: 0.7334
Epoch 7/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5452 - accuracy: 0.7332
Epoch 8/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5439 - accuracy: 0.7341
Epoch 9/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5437 - accuracy: 0.7346
Epoch 10/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5432 - accura

In [28]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5695 - accuracy: 0.7293 - 418ms/epoch - 2ms/step
Loss: 0.5694819092750549, Accuracy: 0.7293294668197632


In [29]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=(len(X_train[0]) * 2), activation='relu', input_dim=len(X_train[0])))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=len(X_train[0]), activation='relu'))
nn.add(tf.keras.layers.Dense(units=len(X_train[0]), activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 94)                4512      
                                                                 
 dense_8 (Dense)             (None, 47)                4465      
                                                                 
 dense_9 (Dense)             (None, 47)                2256      
                                                                 
 dense_10 (Dense)            (None, 1)                 48        
                                                                 
Total params: 11,281
Trainable params: 11,281
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [31]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 4ms/step - loss: 0.5687 - accuracy: 0.7207
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5536 - accuracy: 0.7289
Epoch 3/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5501 - accuracy: 0.7313
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5479 - accuracy: 0.7320
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5467 - accuracy: 0.7331
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5452 - accuracy: 0.7343
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5441 - accuracy: 0.7327
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5432 - accuracy: 0.7338
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5432 - accuracy: 0.7347
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5420 - accura

In [32]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5925 - accuracy: 0.7292 - 568ms/epoch - 2ms/step
Loss: 0.592539370059967, Accuracy: 0.7292128205299377


In [37]:
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=100,
        step=2), activation=activation, input_dim=47))

    for i in range(hp.Int('num_layers', 1, 3)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=100,
            step=2),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [38]:
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=50,
    hyperband_iterations=2)

In [39]:
tuner.search(X_train_scaled,y_train,epochs=50,validation_data=(X_test_scaled,y_test)) 

Trial 180 Complete [00h 02m 09s]
val_accuracy: 0.7317784428596497

Best val_accuracy So Far: 0.7345772385597229
Total elapsed time: 01h 33m 49s
INFO:tensorflow:Oracle triggered exit


In [40]:
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'first_units': 15,
 'num_layers': 2,
 'units_0': 21,
 'units_1': 23,
 'units_2': 71,
 'tuner/epochs': 17,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 1,
 'tuner/round': 0}

In [41]:
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5528 - accuracy: 0.7346 - 504ms/epoch - 2ms/step
Loss: 0.5527530908584595, Accuracy: 0.7345772385597229


In [ ]:
# Making a model with the best hyperparameters
nn = tuner.hypermodel.build(best_hyper)

In [ ]:
# Saving the best model to HDF5 file
nn.save('Models/AlphabetSoupCharity_Optimization.h5')